# EDA Notebook

## Data Understanding

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import stats
import data_preparation as pd

%matplotlib inline

ModuleNotFoundError: No module named 'data_preparation'

In [2]:
# Load Kings county database and preview first few entries
kc_df = pd.read_csv('data/kc_house_data.csv')
kc_df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [3]:
# Taking a look at all the columns, data types, and missing entries
kc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

The data contains 21 total columns, 12 of type int or float, and 6 of type object. It contains:

- `id` column which have a unique id value for each house
- `date` column with the sale date (Years 2014 - 2015)
- `price` column with the price of sale and our target variable
    - <p>mean: \$540296</p>
    - <p>std: \$367368</p>
- `bedrooms`, `bathrooms` and `floors` columns with counts of each in their respective house
- four area variables defining the square footage of living space, lot space, area of house apart from the basement, and area of the basement.
- two area variables describing the living and lot area of the 15 nearest neighbors
- `waterfront` variable of whether the house is on a waterfront (Yes/No)
- `view` describing quality of view from house
    - (None, Fair, Average, Good, Excellent)
- `condition` for the overall maintanence condtion of the house
    - (Poor, Fair, Average, Good, Very Good)
- `grade` describing overall construction and design grade of the house 
    - (Poor, Low, Fair, Low Average, Average, Good, Better, Very Good, Excellent, Luxury, Mansion)
- `yr_built` for when the house was built
- `yr_renovated` for when house was renovated last (if applicable)
- three location columns detailing `zipcode`, latitude (`lat`) and longitude (`long`) of the house.
    - Our data consists of an area covering 70 zipcodes


In [19]:
# Checking the years spanned by the data set
sales_year = kc_df.date.map(lambda x: int(x[-4:]))

print('The data spans the years {} to {}'.format(min(sales_year), max(sales_year)))

# finds mean and standard deviation of price
print('Sales price mean: ${}, stdev: ${}'.format(int(kc_df.price.mean()),int(kc_df.price.std())))

The data spans the years 2014 to 2015
Sales price mean: $540296, stdev: $367368


In [21]:
# Checking view values
kc_df.view.value_counts()

NONE         19422
AVERAGE        957
GOOD           508
FAIR           330
EXCELLENT      317
Name: view, dtype: int64

In [22]:
# Checking condition values
kc_df.condition.value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [23]:
# Checking grade values
kc_df.grade.value_counts()

7 Average        8974
8 Good           6065
9 Better         2615
6 Low Average    2038
10 Very Good     1134
11 Excellent      399
5 Fair            242
12 Luxury          89
4 Low              27
13 Mansion         13
3 Poor              1
Name: grade, dtype: int64

In [20]:
# Checking how many unique zipcodes are included in the dataset
num_of_zip = len(kc_df.zipcode.unique())
print('There are {} unique zipcodes in this data set'.format(num_of_zip))

There are 70 unique zipcodes in this data set


## Data Preparation


At first glance, the columns for `id`, latitude (`lat`), and longitude (`long`) can be removed. The `id` column is good for reference, but not good for a linear regression model. The latitude and longitude appear to be difficult variables to deal with since they describe an exact location. In a linear regression model their seperate coefficients would only be good for how far in a cordinal direction they are. Having zones like a zipcode would be preferable as that defines a region, though having a 70 unique zipcodes in our data set would create a lot of columns if they were treated categorically and one hot encoded. Therefore the `zipcode` column will also be ommitted from our calculations. Another column that can be removed is the `date`. If our data set spanned a longer time this column may be used to adjust for inflation, or observe outside conditions that caused housing prices to drastically change like the 2006 housing bubble. Since our data only spans the years of 2014 and 2015 this isn't necessary.

We also have to consider which columns can actually pertain to our problem. We are looking for the best variables to improve to increase the sales price. Variables like `waterfront`, `view`, `yr_built`, `yr_renovated`, `sqft_living15` and `sqft_lot15` are variables that all depend on location and can't be improved after purchase. Considering that these variables are most likely impossible to change, they will also be ommitted from our calculations.

In [71]:
# Dropping unnecessary columns from original dataset
unn_columns = ['id', 'date', 'lat', 'long', 'zipcode', 'waterfront',
               'view', 'yr_built', 'yr_renovated', 'sqft_living15',
              'sqft_lot15']

kc_df_iprep = kc_df.drop(columns=unn_columns, axis=1).copy()
kc_df_iprep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21597 non-null  float64
 1   bedrooms       21597 non-null  int64  
 2   bathrooms      21597 non-null  float64
 3   sqft_living    21597 non-null  int64  
 4   sqft_lot       21597 non-null  int64  
 5   floors         21597 non-null  float64
 6   condition      21597 non-null  object 
 7   grade          21597 non-null  object 
 8   sqft_above     21597 non-null  int64  
 9   sqft_basement  21597 non-null  object 
dtypes: float64(3), int64(4), object(3)
memory usage: 1.6+ MB


We now need to handle the three columns left of type object. The different entries from `condition` and `grade` were already seen and understood. Since these two columns are rankings and order, they will be treated as numerical columns instead of categorical. 

The `grade` columns values already have a numerical ranking assigned to them followed by the description. Therefore we will just remove the description for each.

The `condition` column only contains the description, so they will be replaced using the map below.

- Poor: 1
- Fair: 2
- Average: 3
- Good: 4
- Very Good: 5

In [72]:
# Taking a look at sqft_basement values
# There were many unique values so value_counts was avoided
objects = {}

for x in kc_df_iprep.sqft_basement:
    try:
        float(x)
    except:
        objects[x] = objects.get(x, 0) + 1
objects

{'?': 454}

In [73]:
kc_df_iprep.sqft_basement.value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2180.0        1
1880.0        1
2720.0        1
666.0         1
2580.0        1
Name: sqft_basement, Length: 304, dtype: int64

In [74]:
med = kc_df_iprep.loc[(kc_df_iprep.sqft_basement != '?'), 'sqft_basement'].map(lambda x: float(x)).median()



The `sqft_basement` data contains a question mark. This is possibly entered when there was a basement with no access. These entries will be replaced with the median value of the rest of the entries which turns out to be zero.

In [75]:
def initial_prep(df):
#     Maps new values for condition column
    condition_map = {'Poor': 1, 'Fair': 2, 'Average': 3, 'Good': 4, 'Very Good': 5}
    df.condition = df.condition.map(condition_map)

#    Function to take only numeric value of grade rating
    def grade_change(ds):
        return int(ds[:2])

#     Takes only numerical value of grade rating    
    df.grade = df.grade.map(grade_change)
    
#     Replaces ? in sqft_basement to the median value, and changes dtype to float
    med = df.loc[(df.sqft_basement != '?'),
             'sqft_basement'].map(lambda x: float(x)).median() 

    def sqft_basement_map(ds):
        try:
            return float(ds)
        except:
            return med

    df.sqft_basement = df.sqft_basement.map(sqft_basement_map)
    
    return df

In [76]:
kc_df_iprep = initial_prep(kc_df_iprep)
kc_df_iprep.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement
0,221900.0,3,1.00,1180,5650,1.0,3,7,1180,0.0
1,538000.0,3,2.25,2570,7242,2.0,3,7,2170,400.0
2,180000.0,2,1.00,770,10000,1.0,3,6,770,0.0
3,604000.0,4,3.00,1960,5000,1.0,5,7,1050,910.0
4,510000.0,3,2.00,1680,8080,1.0,3,8,1680,0.0


## Modeling

The initial data preparation is completed, it's time to start our first model which will be a simple linear regression of the variable with the highest correlation to price.

### Base Model


In [80]:
# To find the column with highest correlation to price
kc_df_iprep.corr()['price'].sort_values(ascending=False)

price            1.000000
sqft_living      0.701917
grade            0.667951
sqft_above       0.605368
bathrooms        0.525906
sqft_basement    0.321108
bedrooms         0.308787
floors           0.256804
sqft_lot         0.089876
condition        0.036056
Name: price, dtype: float64

The variable with the highest correlatin to `price` is `sqft_living`. Let's define these as our x and y variables.

'''
Initial notes on columns:

Dropable Columns: ['id', 'date', 'lat', 'long']

Missing Data or Misunderstood Data (NaN): waterfront, view, yr_renovated
**Need to research document to find more information on NaN values

Data Input: 
    Object Columns: categoricals = ['waterfront', 'view', 'condition', 'grade']
    *** Not sure yet how I want to deal with grade
    
grade: contains numeric and written classification, should get rid of written classification

yr_renovated: has both 0 and NaN inputs, look into this 

waterfront: has both no and NaN

view: has both None and NaN

Data needs to be normalized: Columns including sqft have large values, also some have zero values because they
don't exist in those homes

sqft_lots has some outliers, may need to be cut off
bedrooms has a clear outlier of 33, possibly even 10 and 11
Maybe split year built into segments and categorize them.
'''